# Transformación linear de una variable bivariada

Esta demo muestra la transformación de una variable normal bivariada con correlación $X$ a una variable $Y$ sin correlación. 
Este ejemplo es útil para diagonalizar la forma cuadrática,
$ Q = (\boldsymbol{X}-\boldsymbol{\mu}_X)^T \, \boldsymbol{V}_X^{-1} \, (\boldsymbol{X}-\boldsymbol{\mu}_X)$, que aparece en la PDF conjunta de una variable normal bivariada.

## Variable normal bivariada $X$

$X$ = (altura, peso) de una persona

In [ ]:
import numpy as np
# Height in centimeters
mean_height = 159  
# Weight in kilograms
mean_weight = 62.5
mean_x = np.array([mean_height, mean_weight])
mean_x

Matrix de covarianza de $X$

In [ ]:
import danatools
sigma_height = 10
sigma_weight = 7
correlation = 0.7
covariance_x = danatools.covariance_matrix_2d(sigma_height, sigma_weight, correlation)
covariance_x

Generamos muchos eventos de la variable $X$

In [ ]:
from scipy.stats import multivariate_normal
population = 1000000
rng = np.random.default_rng(seed=6870)
data_x = multivariate_normal.rvs(mean_x, covariance_x, size=population, random_state=rng)
data_x

Plot del histograma de densidad y de las elipses de covarianza de la PDF conjunta de $X$  

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
fig, ax = plt.subplots()
ax.set_xlabel("$X_1$")
ax.set_ylabel("$X_2$")

height_limits = [mean_height-3*sigma_height, mean_height+3*sigma_height]
weight_limits = [mean_weight-3*sigma_weight, mean_weight+3*sigma_weight]
counts, xedges, yedges, im = ax.hist2d(data_x[:,0], data_x[:,1], range=[height_limits, weight_limits], bins=100, density=True, cmap=cm.viridis)
clb = plt.colorbar(im)

ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=1), color='tab:red', ls='--')
ax.plot(*danatools.get_ellipse(mean_x, covariance_x, nsigma=2), color='tab:red', ls='--')

## Variable bivariada $Y$

Obtenemos la variable $Y$ a partir de la diagonalización de la matriz hessiana de $X$

Matriz hessiana 

In [ ]:
from numpy import linalg
hessian_matrix = linalg.inv(covariance_x)

Diagonalización de la matriz hessiana

In [ ]:
eigenvalues, eigenvectors = linalg.eig(hessian_matrix)

In [ ]:
eigenvalues

In [ ]:
eigenvectors

*Nota*: también se puede diagonalizar la matriz de covarianza en lugar de la hessiana ya que una matriz y su inversa tienen los mismos autovectores.

Matriz de rotación

In [ ]:
# Transpossing to put the eigenvectors in the columns of the rotation matrix as required for matrix multiplication 
rotation_matrix = eigenvectors.T
rotation_matrix

Multiplicamos los eventos de $X$ por la matriz de rotación para obtener los eventos de $Y$

In [ ]:
# Using an Einstein sum to transform all X events in a single line of code
data_y = np.einsum("nk,pk->pn", rotation_matrix, data_x)
data_y

In [ ]:
len(data_y)

Media de $Y$

In [ ]:
mean_y = rotation_matrix @ mean_x
mean_y

In [ ]:
mean_y1 = mean_y[0]
mean_y2 = mean_y[1]

Covarianza de $Y$

In [ ]:
covariance_y = rotation_matrix @ covariance_x @ rotation_matrix.T
covariance_y

La matriz de covarianza de $Y$ es diagonal *dentro de la precisión númerica*

In [ ]:
import math
sigma_y1 = math.sqrt(covariance_y[0,0])
sigma_y1

In [ ]:
sigma_y2 = math.sqrt(covariance_y[1,1])
sigma_y2

Plot del histograma de densidad y de las elipses de covarianza de la PDF conjunta de $Y$  

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("$Y_1$")
ax.set_ylabel("$Y_2$")

y1_limits = [mean_y1-3*sigma_y1, mean_y1+3*sigma_y1]
y2_limits = [mean_y2-5*sigma_y2, mean_y2+5*sigma_y2]
counts, xedges, yedges, im = ax.hist2d(data_y[:,0], data_y[:,1], range=[y1_limits, y2_limits], bins=100, density=True, cmap=cm.viridis)
clb = plt.colorbar(im)

import danatools
ax.plot(*danatools.get_ellipse(mean_y, covariance_y, nsigma=1), color='tab:red', ls='--')
ax.plot(*danatools.get_ellipse(mean_y, covariance_y, nsigma=2), color='tab:red', ls='--')

La base de autovectores se obtiene mediante una rotación de la base canónica
En la base de autovectores de la matriz Hessiana, la matriz de covarianza de $Y$ es diagonal.
Por lo tanto la correlación entre las dos componentes de $Y$ es nula.
Como $Y$ sigue una distribución normal bivariada, correlación nula implica independencia de las componentes de $Y$. 

Esta demo muestra que una variable con correlación $X$ se puede reducir a una variable sin correlación $Y$ a través de una rotación de la  base de vectores 